In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, RidgeCV, Lasso

In [2]:
df=pd.read_csv('/rds/general/user/hsl121/home/hda_project/rq1/rq1_cleaned_no_ae.csv')
adverse=pd.read_csv('/rds/general/user/hsl121/home/hda_project/rq1/rq1_full_df_adverse.csv')

In [23]:
df.shape

(20593, 106)

Binary AE Added

In [3]:
df = df.merge(adverse[['SID', 'adverse_binary']], on='SID', how='left')

In [27]:
df.shape

(20593, 107)

In [4]:
df.columns
drop=['EQ5D_Round2', 'EQ5D_Round3', 'EQ5D_Round4','EQ5D_Round5', 'EQ5D_Round6']
df=df.drop(columns=drop)

In [5]:
full=df.copy()

In [6]:
X=full.drop(['adverse_binary','SID'], axis=1)
y=full['adverse_binary']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [39]:
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.3f}")
print(f"Test R²: {r2:.3f}")


Test MSE: 0.052
Test R²: 0.022


XGBoost

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    scale_pos_weight=(y == 0).sum() / (y == 1).sum()  # handle imbalance
)

xgb_model.fit(X_train, y_train)


/rds/general/user/hsl121/home/anaconda3/envs/TDS/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:22:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
y_pred = xgb_model.predict(X_test)
y_proba = xgb_model.predict_proba(X_test)[:, 1]

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))
